# MiniCPM API Query Example

This notebook demonstrates how to use the MiniCPM API with two images.

In [2]:
import requests
import base64
import json
from PIL import Image
import io

## Prepare the images

In [33]:
def image_to_base64(image_path):
    with Image.open(image_path) as img:
        img = img.convert('RGB')
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Replace these with the paths to your actual images
image1_path = "test_images/adxl.png"
image2_path = "test_images/adxl1.png"

image1_base64 = image_to_base64(image1_path)
image2_base64 = image_to_base64(image2_path)

## Prepare the API request

In [38]:
api_url = "http://127.0.0.1:8001/v1/chat/completions"  # Update this if your API is hosted elsewhere

payload = {
    "model": "MiniCPM-V-2_6",  # This might not be used by your API, but included for compatibility
    "temperature": 0.1,
    "messages": [
        {
          "role":"system",
          "content":"You are a good engineer"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image1_base64}"
                    }
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image2_base64}"
                    }
                },
                {
                    "type": "text",
                    "text": "Compare image 1 and image 2, tell me about the differences between them. Format your response in JSON."
                }
            ]
        }
    ],
    "seed": 4,
    "response_format": {
        "type": "json_object"
    },
    "max_tokens": 4095
}

## Send the API request

In [39]:
response = requests.post(api_url, json=payload)

if response.status_code == 200:
    result = response.json()
    print(json.dumps(result, indent=2))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{
  "id": "7a247d6e-4d3b-4b77-a083-b525c664be58",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the provided images, here are the differences between image 1 and image 2:\n\n1. **Page Number**:\n   - Image 1: Page number is not visible.\n   - Image 2: Page number \"4 of 36\" is visible at the bottom right corner.\n\n2. **Content Differences**:\n   - The content in both images appears to be identical with respect to text and layout. There are no visible changes or additions in the textual information presented in both images.\n\n3. **Footer Information**:\n   - Both images have a footer that includes the website \"analog.com\".\n   - Both images have a revision control note \"Rev. G\".\n\nIn summary, the primary difference between the two images is the visibility of the page number in image 2, which indicates it is part of a larger document (36 pages).",
        "refusal": null,
        "tool_calls": null,
        "

## Parse and display the result

In [ ]:
if response.status_code == 200:
    result = response.json()
    if 'response' in result:
        try:
            parsed_response = json.loads(result['response'])
            print(json.dumps(parsed_response, indent=2))
        except json.JSONDecodeError:
            print("The response is not in valid JSON format:")
            print(result['response'])
    else:
        print("Unexpected response format:")
        print(json.dumps(result, indent=2))